## Imports


In [1]:
import cv2
from typing import Tuple, List
from math import sin, cos, pi
from skimage.draw import line as line_points
import numpy as np
from numpy.linalg import norm

c:\Users\Peter\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## Constants


In [2]:
RED = (0, 0, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)
BLACK = (0, 0, 0)
RAD2DEG = 180 / pi
DEG2RAD = pi / 180
FILL = -1
MAX_RANGE = 1200
SENSOR_ANGLE = 250  # (-125,125)
SENSOR_STEP = 2

## Functions


In [52]:
def convert_from_raduis_to_cartesian(
    x: int, y: int, r: float, theta: float
) -> Tuple[float, float]:
    """
    Convert from raduis and theta to cartesian coordinates
    """
    x2 = int(x + r * cos(theta * DEG2RAD))
    y2 = int(y + r * sin(theta * DEG2RAD))

    return (x2, y2)


def convert_from_raduis_to_cartesian_end_point(
    x: int, y: int, r: float, theta: float

) -> Tuple[float, float]:
    """

    Convert from raduis and theta to cartesian coordinates
    """

    x2 = round(x + r * cos(theta * DEG2RAD))

    y2 = round(y + r * sin(theta * DEG2RAD))

    return (x2, y2)

## Requirement 1


In [41]:
def req1(image_path: str, x_robot: int, y_robot: int, theta_robot: int):
    # read the image using opencv
    img = cv2.imread(image_path)
    # draw red cricle at the robot position which is (x,y,theta) = (350,185,200)
    cv2.circle(img, (x_robot, y_robot), 10, RED, FILL)
    # draw the robot direction
    cv2.line(
        img,
        (x_robot, y_robot),
        convert_from_raduis_to_cartesian(x_robot, y_robot, 10, theta_robot),
        BLUE,
        thickness=2,
    )
    # open the measurements file
    # write the position of the robot in txt file
    file = open("observations.txt", "w")
    file.write(
        f"Robot position: x= {str(x_robot)} , y= {str(y_robot)} , theta= {str(theta_robot)}"
    )

    # write meaeurements in txt file
    file.write("\nObservations: ")
    file.write("\nAngle \t Distance(cm) \t Endpoint")

    # create array to save the distance at each angle
    distances = np.zeros((SENSOR_ANGLE // SENSOR_STEP) + 1)

    # list of the locations of the endpoints at each angle
    endpoints = []
    # loop over the sensor angles
    for ind, angle in enumerate(
        range(
            theta_robot - (SENSOR_ANGLE // SENSOR_STEP),
            theta_robot + (SENSOR_ANGLE // SENSOR_STEP) + 1,
            SENSOR_STEP,
        )
    ):
        # get the end of the ray
        x_max, y_max = convert_from_raduis_to_cartesian(
            x_robot, y_robot, MAX_RANGE // 4, angle
        )
        # get all the points in the line
        points_in_line = line_points(
            x_robot,
            y_robot,
            x_max,
            y_max,
        )
        x_points = points_in_line[0]
        y_points = points_in_line[1]
        # get the final point in the line
        x_final = x_points[0]
        y_final = y_points[0]
        # loop over the points in the line
        for i in range(len(x_points)):
            x = x_points[i]
            y = y_points[i]
            if all(img[y, x] == BLACK):
                break
            else:
                x_final = x
                y_final = y

        endpoint = (x_final, y_final)
        # draw a line from the robot to the final point
        cv2.line(
            img,
            (x_robot, y_robot),
            endpoint,
            GREEN,
            thickness=2,
        )
        # calculate the distance between the robot and the endpoint
        distance = norm(np.array((x_robot, y_robot)) - np.array(endpoint))
        # save the distance in the array
        distances[ind] = min(MAX_RANGE, distance * 4)
        endpoints.append(endpoint)

        # write meaeurements in txt file
        file.write(f"\n {str(angle)} \t {str(distances[ind])} \t {str(endpoint)}")
    # save the image
    cv2.imwrite("req1.jpg", img)
    # close the file
    file.close()
    # show the image
    # cv2.imshow("image", img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # return distances, np.array(endpoints)

In [42]:
req1("./Map.jpg", 350, 185, 200)

## Requirement 2


In [ ]:
def req2(image_path: str, sensor_range: float, variance: float = 1):
    """
    snesor_range: the maximum distance that the sensor can measure
    variance: the variance of the gaussian distribution
    """
    # read the map image and convert it to gray scale
    map = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # do binary thresholding
    binary_map = cv2.threshold(map, 127, 255, cv2.THRESH_BINARY)[1]

    # for each pixel in the map calculate the distance to the nearest obstacle
    """
    The distanceTransform function in OpenCV is an image processing function that calculates
    the Euclidean distance between each non-zero pixel in an image and the nearest zero pixel
    """
    distances_to_nearest_obst = cv2.distanceTransform(binary_map, cv2.DIST_L2, 0)
    distances_to_nearest_obst = np.array(distances_to_nearest_obst)
    # calculate the likelihood field using gaussian distribution with mean = 0 and variance = variance
    likelihood_field = (
        1
        / (np.sqrt(2 * np.pi * variance))
        * np.exp(-(distances_to_nearest_obst**2) / (2 * variance))
    ) + 0.0001
    # max likelihood
    max_likelihood = np.max(likelihood_field)
    # # normalize the likelihood field
    likelihood_field = likelihood_field / max_likelihood
    # output the likelihood field
    cv2.imwrite(f"likelihood_field for var={variance}.jpg", likelihood_field * 255)

    # probability map
    probability_map = np.zeros(map.shape)
    max_probability = 0
    index_max_probability = (0, 0)
    # loop over the likelihood field
    for y in range(likelihood_field.shape[0]):
        for x in range(likelihood_field.shape[1]):
            # array to save the probability at each angle
            probabilities = np.zeros(361)
            for angle in range(361):
                # get the endpoint of the ray
                x_max, y_max = convert_from_raduis_to_cartesian_end_point(
                    x, y, sensor_range / 4, angle
                )
                # if the endpoint is out of the map, then give it a very low probability ( 0.0001 )
                if (
                    x_max < 0
                    or x_max >= likelihood_field.shape[1]
                    or y_max < 0
                    or y_max >= likelihood_field.shape[0]
                ):
                    probability = 0.0001
                else:
                    # get the probability from the likelihood field
                    probability = likelihood_field[y_max, x_max]
                # save the probability in the array
                probabilities[angle] = probability
            # the probability of the pixel is the maximum probability at all angles
            probability_map[y, x] = np.max(probabilities)
    # save the probability map
    cv2.imwrite(
        f"probability_map sensor_range={sensor_range} var={variance}.jpg",
        probability_map * 255,
    )
    # get the point with the highest probability with reverse order
    index_max_probability = np.unravel_index(
        np.argmax(probability_map), probability_map.shape
    )
    max_probability = probability_map[index_max_probability]
    # print the value of the point with the highest probability
    print(f"The highest probability is: {max_probability} at {index_max_probability}")

    # draw the point with the highest probability in the map
    # max_point_map = np.zeros(map.shape)
    # max_point_map[index_max_probability] = 1
    # cv2.imwrite(
    #     f"max_point_map sensor_range={sensor_range} var={variance}.jpg",
    #     max_point_map * 255,
    # )

    # show the image
    # cv2.imshow(f"likelihood_field for var={variance}", likelihood_field * 255)
    # cv2.imshow(f"probability_map sensor_range={sensor_range} var={variance}", probability_map * 255)
    # cv2.imshow(f"max_point_map sensor_range={sensor_range} var={variance}", max_point_map * 255)

    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [63]:
for sensor_range_meter in [0.1,0.5,1,2,3,4]:
    for std in [1,5,10,20]:
        req2("./Map.jpg", sensor_range_meter * 100, std**2)

The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at (0, 0)
The highest probability is: 1.0 at